In [11]:
import csv
import tensorflow as tf
import numpy as np
import json
import pickle

In [12]:
pre_datas_test="/home/admin/xssdetection/file/pre_datas_test.csv"
vec_dir="/home/admin/xssdetection/file/word2vec.pickle"
def data_generator(data_dir):
    reader = tf.TextLineReader()
    queue = tf.train.string_input_producer([data_dir])
    _, value = reader.read(queue)
    coord = tf.train.Coordinator()
    sess = tf.Session()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    while True:
        v = sess.run(value)
        [data, label] = v.split(b"|")
        data = np.array(json.loads(data.decode("utf-8")))
        label = np.array(json.loads(label.decode("utf-8")))
        yield (data, label)
    coord.request_stop()
    coord.join(threads)
    sess.close()


In [13]:
generator=data_generator(pre_datas_test)
data,label=next(generator)

In [14]:
data.shape

(532,)

In [15]:
batch_data = []
data_embed = []
with open(vec_dir,"rb") as f :
    word2vec=pickle.load(f)
    dictionary=word2vec["dictionary"]
    reverse_dictionary=word2vec["reverse_dictionary"]
    embeddings=word2vec["embeddings"]
for d in data:
    if d != -1:
        data_embed.append(embeddings[reverse_dictionary[d]])
    else:
        data_embed.append([0.0] * len(embeddings["UNK"]))
batch_data.append(data_embed)
test_data=np.array(batch_data)

In [24]:
display(test_data.shape)

(1, 532, 128)

In [17]:
from keras.models import load_model
model=load_model('./file/xss_lstm.h5')

In [18]:
#batch data(batch_size=500) testing,set steps=1
model.predict(test_data,steps=1)[0]

array([0.99782795, 0.00217206], dtype=float32)

In [20]:
#single data testing
from keras.models import load_model
model2=load_model('./file/xss_lstm_5000.h5')

In [21]:
model2.predict(test_data)

array([[0.9816626 , 0.01833741]], dtype=float32)